In [1]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
import yaml
import re
import math

import gbd_mapping as gm
from vivarium import Artifact

from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
from get_draws.api import get_draws

import vivarium_helpers as vh
import vivarium_helpers.id_helper as idh
from vivarium_helpers.vph_output.operations import VPHOperator, summarize_draws
from vivarium_helpers.utils import aggregate_mean_lower_upper, convert_to_categorical, print_memory_usage

!date
!whoami
!pwd

Thu Nov 13 00:31:41 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/scratch


In [2]:
# Project directory
project_dir = '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/'

# For testing: Run directory containing model 8.3 results for all
# locations
model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# Results directory for model 8.3, for testing
results_dir = Path(project_dir) / model_run_subdir / 'results/'

# Artifact for models 8.3 and 8.4
artifact_model_number = '8.3'

print(results_dir.exists())
results_dir

True


PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/results')

In [7]:
!ls ../results_tables/syl

2025_10_28_csf_pet_tests_final.csv
2025_10_28_dalys_final.csv
2025_10_28_deaths_final.csv
2025_10_28_incidence_final.csv
2025_10_28_medication_final.csv
2025_10_28_prevalence_final.csv
2025_10_28_results_dataframe_health.ipynb
2025_10_28_results_dataframe_test_treat.ipynb


In [9]:
incidence = pd.read_csv('../results_tables/syl/2025_10_28_incidence_final.csv', index_col=0)
incidence

,Year ID,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,Incident cases,Number,0.0,0.0,0.0
1,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,Incident cases,"Rate per 100,000",0.0,0.0,0.0
2,2025,Brazil,25_to_29,Female,MCI due to AD,BBBM Testing Only,Incident cases,Number,0.0,0.0,0.0
3,2025,Brazil,25_to_29,Female,MCI due to AD,BBBM Testing Only,Incident cases,"Rate per 100,000",0.0,0.0,0.0
4,2025,Brazil,25_to_29,Female,MCI due to AD,BBBM Testing and Treatment,Incident cases,Number,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
38875,2060,United States of America,95_plus,Male,Preclinical AD,Reference,Incident cases,"Rate per 100,000",0.0,0.0,0.0
38876,2060,United States of America,95_plus,Male,Preclinical AD,BBBM Testing Only,Incident cases,Number,0.0,0.0,0.0
38877,2060,United States of America,95_plus,Male,Preclinical AD,BBBM Testing Only,Incident cases,"Rate per 100,000",0.0,0.0,0.0
38878,2060,United States of America,95_plus,Male,Preclinical AD,BBBM Testing and Treatment,Incident cases,Number,0.0,0.0,0.0


In [10]:
incidence['Disease Stage'].unique()

array(['MCI due to AD', 'AD Dementia', 'Preclinical AD'], dtype=object)

In [11]:
incidence['Location'].unique()

array(['Brazil', 'United States of America'], dtype=object)

# Let's count how many rows we should have per measure

Columns are:

* location (10 locations)
* year (2025 - 2100)
* age group (25-29, ..., 90-94, 95+)
* sex (male, female, both)
* disease stage (preclinical, MCI, dementia)
* scenario (3 scenarios)
* metric (number, rate)
* measure (lots of measures)


If all strata are included, the total number of rows _per measure_ is
615,600. So, large, but not insane.


In [12]:
strata_counts = {
    'location': 10,
    'year': len(range(2025, 2101)),
    'age_group': len(range(25, 95, 5)) + 1, # add 1 for 95+
    'sex': 3,
    'disease_stage': 3,
    'scenario': 3,
    'metric': 2
}
strata_counts

{'location': 10,
 'year': 76,
 'age_group': 15,
 'sex': 3,
 'disease_stage': 3,
 'scenario': 3,
 'metric': 2}

In [13]:
math.prod(strata_counts.values())

615600

In [14]:
strata_counts_2 = {
    'location': 10,
    'year': len(range(2022, 2101)),
    'age_group': len(range(25, 95, 5)) + 1, # add 1 for 95+
    'sex': 2,
    'disease_stage': 3,
    'scenario': 3,
    'metric': 1
}
strata_counts_2

{'location': 10,
 'year': 79,
 'age_group': 15,
 'sex': 2,
 'disease_stage': 3,
 'scenario': 3,
 'metric': 1}

In [15]:
math.prod(strata_counts_2.values())

213300

# Test renaming things with Categoricals

It looks like using `DataFrame.replace` on Categorical columns,
preserves the Categoricals.

In [16]:
rng = np.random.default_rng(555666777)
n = 10
df = pd.DataFrame(
    {
        'species': rng.choice(['yak', 'smurf'], n),
        'sex': rng.choice(['male', 'female'], n),
    }
).astype('category')
df

,species,sex
0,smurf,male
1,smurf,male
2,smurf,female
3,yak,male
4,yak,male
5,yak,female
6,yak,female
7,smurf,male
8,yak,male
9,yak,male


In [17]:
df['species']

0    smurf
1    smurf
2    smurf
3      yak
4      yak
5      yak
6      yak
7    smurf
8      yak
9      yak
Name: species, dtype: category
Categories (2, object): ['smurf', 'yak']

In [18]:
df2 = df.replace({'yak': 'squid'})
df2

,species,sex
0,smurf,male
1,smurf,male
2,smurf,female
3,squid,male
4,squid,male
5,squid,female
6,squid,female
7,smurf,male
8,squid,male
9,squid,male


In [19]:
df2['species']

0    smurf
1    smurf
2    smurf
3    squid
4    squid
5    squid
6    squid
7    smurf
8    squid
9    squid
Name: species, dtype: category
Categories (2, object): ['smurf', 'squid']

In [20]:
df['species'].replace({'smurf': 'goat'})

0    goat
1    goat
2    goat
3     yak
4     yak
5     yak
6     yak
7    goat
8     yak
9     yak
Name: species, dtype: category
Categories (2, object): ['goat', 'yak']

In [21]:
df3 = df.replace({'species': {'smurf': 'orc', 'yak': 'orc'}})
df3['species']

0    orc
1    orc
2    orc
3    orc
4    orc
5    orc
6    orc
7    orc
8    orc
9    orc
Name: species, dtype: category
Categories (1, object): ['orc']

# Test `str.replace` with Categoricals

It looks like using `.str.replace` on Categorical
columns converts their dtype to `object`.

In [ ]:
# Calling .str.replace converte back to object
df.species.str.replace('u', 'o')

0    smorf
1    smorf
2    smorf
3      yak
4      yak
5      yak
6      yak
7    smorf
8      yak
9      yak
Name: species, dtype: object

In [26]:
df.species.cat.categories.str.replace('u', 'o')

Index(['smorf', 'yak'], dtype='object')

In [27]:
def cat_str_replace(s, *args, **kwargs):
    new_categories = s.cat.categories.str.replace(*args, **kwargs)
    return s.cat.rename_categories(new_categories)

cat_str_replace(df.species, 'u', 'o')

0    smorf
1    smorf
2    smorf
3      yak
4      yak
5      yak
6      yak
7    smorf
8      yak
9      yak
Name: species, dtype: category
Categories (2, object): ['smorf', 'yak']

In [43]:
rng.integers(5, size=10)

array([3, 2, 1, 2, 3, 2, 4, 3, 3, 4])

In [ ]:
# NOTE:Fails with ValueError: Categorical categories must be unique
# We can fix it using the merge_categories function below
# cat_str_replace(df.sex, 'fe', '')

In [125]:
def cat_str_replace2(s, *args, **kwargs):
    new_categories = s.cat.categories.str.replace(*args, **kwargs)
    category_map = dict(zip(s.cat.categories, new_categories))
    return s.replace(category_map)

cat_str_replace2(df.species, 'u', 'o')

0    smorf
1    smorf
2    smorf
3      yak
4      yak
5      yak
6      yak
7    smorf
8      yak
9      yak
Name: species, dtype: category
Categories (2, object): ['smorf', 'yak']

In [ ]:
# It looks like .map preserves categoricals when the map is 1-to-1
df.species.map({'smurf': 'smorf', 'yak': 'yak'})

0    smorf
1    smorf
2    smorf
3      yak
4      yak
5      yak
6      yak
7    smorf
8      yak
9      yak
Name: species, dtype: category
Categories (2, object): ['smorf', 'yak']

In [118]:
cat_str_replace2(df.sex, 'fe', '')

0    male
1    male
2    male
3    male
4    male
5    male
6    male
7    male
8    male
9    male
Name: sex, dtype: category
Categories (1, object): ['male']

In [ ]:
# It looks like .map does NOT preserve categoricals when the map is not
# 1-to-1
df.sex.map({'female': 'male', 'male': 'male'})

0    male
1    male
2    male
3    male
4    male
5    male
6    male
7    male
8    male
9    male
Name: sex, dtype: object

In [ ]:
# But .replace still preserves categoricals for many-to-1 maps
df.sex.replace({'female': 'male', 'male': 'male'})

0    male
1    male
2    male
3    male
4    male
5    male
6    male
7    male
8    male
9    male
Name: sex, dtype: category
Categories (1, object): ['male']

In [119]:
df.replace({'sex': {'female': 'male', 'male': 'male'}}).sex

0    male
1    male
2    male
3    male
4    male
5    male
6    male
7    male
8    male
9    male
Name: sex, dtype: category
Categories (1, object): ['male']

In [102]:
df.sex.str.split('m')

0      [, ale]
1      [, ale]
2    [fe, ale]
3      [, ale]
4      [, ale]
5    [fe, ale]
6    [fe, ale]
7      [, ale]
8      [, ale]
9      [, ale]
Name: sex, dtype: object

# Time the two versions of the function in a case where they both work

The version using .replace takes twice is long, but should always be correct.

In [131]:
%timeit cat_str_replace(df.species, 'u', 'o')

326 μs ± 26.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [132]:
%timeit cat_str_replace2(df.species, 'u', 'o')

693 μs ± 7.06 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Test `str.extract` with Categoricals

In [44]:
sentences = df.species.cat.rename_categories(lambda s: f'eat {rng.integers(10)} {s} snacks')
sentences

0    eat 4 smurf snacks
1    eat 4 smurf snacks
2    eat 4 smurf snacks
3      eat 9 yak snacks
4      eat 9 yak snacks
5      eat 9 yak snacks
6      eat 9 yak snacks
7    eat 4 smurf snacks
8      eat 9 yak snacks
9      eat 9 yak snacks
Name: species, dtype: category
Categories (2, object): ['eat 4 smurf snacks', 'eat 9 yak snacks']

In [46]:
extraction_pattern = r"^eat (?P<number>\d) (?P<species>\w+) snacks$"
extracted = sentences.str.extract(extraction_pattern)
extracted

,number,species
0,4,smurf
1,4,smurf
2,4,smurf
3,9,yak
4,9,yak
5,9,yak
6,9,yak
7,4,smurf
8,9,yak
9,9,yak


In [47]:
extracted.dtypes

number     object
species    object
dtype: object

In [68]:
extracted.number

0    4
1    4
2    4
3    9
4    9
5    9
6    9
7    4
8    9
9    9
Name: number, dtype: object

In [72]:
print(extracted.species.equals(df.species))
print(extracted.species.astype('category').equals(df.species))

False
True


In [59]:
def cat_str_extract(s, *args, **kwargs):
    extracted_cats = s.cat.categories.str.extract(*args, **kwargs)
    extracted_cols = [s.cat.rename_categories(col.array).rename(name)
                      for name, col in extracted_cats.items()]
    extracted_df = pd.concat(extracted_cols, axis=1)
    return extracted_df

cat_extracted = cat_str_extract(sentences, extraction_pattern)
cat_extracted

,number,species
0,4,smurf
1,4,smurf
2,4,smurf
3,9,yak
4,9,yak
5,9,yak
6,9,yak
7,4,smurf
8,9,yak
9,9,yak


In [60]:
cat_extracted.dtypes

number     category
species    category
dtype: object

In [65]:
cat_extracted.number

0    4
1    4
2    4
3    9
4    9
5    9
6    9
7    4
8    9
9    9
Name: number, dtype: category
Categories (2, object): ['4', '9']

In [69]:
cat_extracted.species.equals(df.species)

True

In [149]:
cat_extracted.number.replace({'4': 4, '9': 9})

0    4
1    4
2    4
3    9
4    9
5    9
6    9
7    4
8    9
9    9
Name: number, dtype: category
Categories (2, int64): [4, 9]

In [ ]:
# This should do the same thing as my `convert_category_dtype` function:
# https://github.com/ihmeuw/vivarium_research_prl/blob/6450c432fd23b17b580b00fe72a2bb17173ce974/src/vivarium_research_prl/datatypes.py#L141
s = cat_extracted.number
s.replace(s.cat.categories, s.cat.categories.astype('int'))

0    4
1    4
2    4
3    9
4    9
5    9
6    9
7    4
8    9
9    9
Name: number, dtype: category
Categories (2, int64): [4, 9]

In [75]:
sentences2 = pd.Series(
    [f"eat {rng.integers(2, 5)} {species} snacks" for species in df.species.array]
).astype('category')
sentences2

0    eat 2 smurf snacks
1    eat 2 smurf snacks
2    eat 3 smurf snacks
3      eat 2 yak snacks
4      eat 2 yak snacks
5      eat 4 yak snacks
6      eat 3 yak snacks
7    eat 3 smurf snacks
8      eat 4 yak snacks
9      eat 4 yak snacks
dtype: category
Categories (5, object): ['eat 2 smurf snacks', 'eat 2 yak snacks', 'eat 3 smurf snacks', 'eat 3 yak snacks', 'eat 4 yak snacks']

In [ ]:
# This only works with no -1s
o = np.array([1, 3, 0, 2, 1, 0]) # old codes
# m represents {0: 2, 1: 0, 2: 1, 3: 0}
m = np.array([2, 0, 1, 0]) # map from old codes to new codes
n = np.array([0, 0, 2, 1, 0, 2])
m[o] # This maps the array of old cods to the new codes

array([0, 0, 2, 1, 0, 2, 0])

In [145]:
dict(zip(range(4), m))

{0: 2, 1: 0, 2: 1, 3: 0}

In [ ]:
# Fails with ValueError: Categorical categories must be unique
# cat_extracted2 = cat_str_extract(sentences2, extraction_pattern)
# So let's try modifying the function.

# Function copied from:
# https://github.com/ihmeuw/vivarium_research_prl/blob/6450c432fd23b17b580b00fe72a2bb17173ce974/src/vivarium_research_prl/datatypes.py#L107
def merge_categories(categorical: pd.Categorical, old_cat_to_new_cat):
    """Merge the categories in a pandas Categorical according to the mapping
    old_cat_to_new_cat, which can be a function, dict, or Series, and is passed
    to categorical.categories.map. If the mapping is one-to-one, the categories
    will simply be renamed according to the mapping using categorical.rename_categories.
    Otherwise, categorical.categories is replaced with a new array containing
    the unique categories resulting from the mapping, and the old categories
    are mapped accordingly. If s is a Series of type 'category', calling this function
    on s.cat produces the same result as pandas.Categorical(s.map(old_cat_to_new_cat)),
    but it doesn't convert back to a non-categorical Series as an intermediate
    step, hence typically uses less memory and is faster when the number of categories
    is much smaller than the length of the Series.
    Returns a new Categorical with the merged categories.
    """
    new_cat_array = categorical.categories.map(old_cat_to_new_cat)
    new_cats = new_cat_array.unique()
    if len(new_cats) == len(new_cat_array):
        # one-to-one mapping -> no merging is necessary, just renaming of categories
        # Note: Index.unique() returns values in order of appearance, not sorted,
        # therefore new_cats is guaranteed to equal new_cat_array
        new_categorical = categorical.rename_categories(new_cats)
    else:
        # Map each new category to its index in the categories array, i.e., its code
        new_cat_to_new_code = dict(zip(new_cats, range(len(new_cats))))
        # This array replaces each old category with the index (code) of the new category
        new_code_array = new_cat_array.map(new_cat_to_new_code)
        # The index (code) of the old category is mapped to the index (code) of the new category
        old_code_to_new_code = dict(zip(range(len(new_code_array)), new_code_array))
        # -1 indicates NaN and needs to stay the same in the new codes
        old_code_to_new_code[-1] = -1
        new_codes = categorical.codes.map(old_code_to_new_code)
        new_categorical = pd.Categorical.from_codes(new_codes, new_cats)
    return new_categorical

def cat_str_extract2(s, *args, **kwargs):
    extracted_cats = s.cat.categories.str.extract(*args, **kwargs)
    # NOTE: old_cat_to_new_cat = dict(zip(s.cat.categories, col.array)))
    # NOTE: merge_categories returns a Categorical, which can't be
    # concat'ed, so you need to explicitly construct a Series.
    # TODO: It would be more convenient if merge_categories accepted a
    # Series or Categorical and the return type matched the input.
    extracted_cols = [pd.Series(merge_categories(s.cat, dict(zip(s.cat.categories, col.array))), index=s.index, name=name)
                      for name, col in extracted_cats.items()]
    extracted_df = pd.concat(extracted_cols, axis=1)
    # extracted_df = extracted_cats
    return extracted_df

def cat_str_extract3(s, *args, **kwargs):
    extracted_cats = s.cat.categories.str.extract(*args, **kwargs)
    # NOTE: old_cat_to_new_cat = dict(zip(s.cat.categories, col.array)))
    # NOTE: It looks like .replace does the same thing as
    # merge_categories, but can do more as well.
    extracted_cols = [s.replace(dict(zip(s.cat.categories, col.array))).rename(name)
                      for name, col in extracted_cats.items()]
    extracted_df = pd.concat(extracted_cols, axis=1)
    # extracted_df = extracted_cats
    return extracted_df

cat_extracted2 = cat_str_extract2(sentences2, extraction_pattern)
cat_extracted2

,number,species
0,2,smurf
1,2,smurf
2,3,smurf
3,2,yak
4,2,yak
5,4,yak
6,3,yak
7,3,smurf
8,4,yak
9,4,yak


In [92]:
cat_extracted2.number

0    2
1    2
2    3
3    2
4    2
5    4
6    3
7    3
8    4
9    4
Name: number, dtype: category
Categories (3, object): ['2', '3', '4']

In [108]:
cat_extracted2.number.str.isdigit()

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: number, dtype: bool

In [128]:
cat_extracted3 = cat_str_extract3(sentences2, extraction_pattern)
cat_extracted3

,number,species
0,2,smurf
1,2,smurf
2,3,smurf
3,2,yak
4,2,yak
5,4,yak
6,3,yak
7,3,smurf
8,4,yak
9,4,yak


In [129]:
cat_extracted3.species

0    smurf
1    smurf
2    smurf
3      yak
4      yak
5      yak
6      yak
7    smurf
8      yak
9      yak
Name: species, dtype: category
Categories (2, object): ['smurf', 'yak']

# Time `merge_categories` vs. `replace`

In [137]:
%timeit pd.Series(merge_categories(df['species'].cat, {'smurf': 'orc', 'yak': 'orc'}))

2.34 ms ± 139 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [139]:
%timeit df['species'].replace({'smurf': 'orc', 'yak': 'orc'})

967 μs ± 5.98 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [141]:
%timeit cat_str_extract2(sentences2, extraction_pattern)

5.55 ms ± 397 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [142]:
%timeit cat_str_extract3(sentences2, extraction_pattern)

5.86 ms ± 409 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Test aggregation

In [10]:
!ls $results_dir

counts_baseline_tests_among_eligible.parquet
counts_bbbm_tests.parquet
counts_new_simulants.parquet
counts_newly_eligible_for_bbbm_testing.parquet
deaths.parquet
person_time_alzheimers_disease_and_other_dementias.parquet
person_time_eligible_for_bbbm_testing.parquet
person_time_ever_eligible_for_bbbm_testing.parquet
person_time_treatment.parquet
transition_count_alzheimers_disease_and_other_dementias.parquet
transition_count_treatment.parquet
ylds.parquet
ylls.parquet


In [31]:
ops = VPHOperator(location_col=True)

deaths_filters = [
    ('entity', '=', 'alzheimers_disease_state'),
    ('event_year', '>=', '2040'),
    ('event_year', '<', '2050'),
    ('age_group', '>=', '60_to_64'),
    ('age_group', '<', '80_to_84'),
]
deaths = (
    pd.read_parquet(results_dir / 'deaths.parquet', filters=deaths_filters)
    .drop(columns='input_draw_number')
    .pipe(convert_to_categorical)
    .pipe(ops.marginalize, 'random_seed')
)
print_memory_usage(deaths)
deaths

0.2244 MB 


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value
0,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,392,deaths,baseline,Female,alzheimers_disease_state,22.0
1,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,392,deaths,baseline,Male,alzheimers_disease_state,13.0
2,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,392,deaths,bbbm_testing,Female,alzheimers_disease_state,22.0
3,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,392,deaths,bbbm_testing,Male,alzheimers_disease_state,13.0
4,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,392,deaths,bbbm_testing_and_treatment,Female,alzheimers_disease_state,22.0
...,...,...,...,...,...,...,...,...,...,...,...
11995,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,346,deaths,baseline,Male,alzheimers_disease_state,743.0
11996,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,346,deaths,bbbm_testing,Female,alzheimers_disease_state,1711.0
11997,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,346,deaths,bbbm_testing,Male,alzheimers_disease_state,743.0
11998,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,346,deaths,bbbm_testing_and_treatment,Female,alzheimers_disease_state,1694.0


In [26]:
deaths.random_seed.unique()

[6810, 2284, 5616, 2787, 4344]
Categories (5, int64): [2284, 2787, 4344, 5616, 6810]

In [24]:
deaths.event_year.unique()

['2040', '2041', '2042', '2043', '2044', ..., '2056', '2057', '2058', '2059', '2060']
Length: 21
Categories (36, object): ['2025', '2026', '2027', '2028', ..., '2057', '2058', '2059', '2060']

In [36]:
deaths_summary1 = ops.summarize_draws(deaths).reset_index()
deaths_summary1

,age_group,artifact_path,entity,entity_type,event_year,measure,scenario,sex,sub_entity,mean,lower,upper
0,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,baseline,Female,alzheimers_disease_state,29.0,22.7,32.0
1,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,baseline,Male,alzheimers_disease_state,13.0,7.4,21.1
2,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,bbbm_testing,Female,alzheimers_disease_state,29.0,22.7,32.0
3,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,bbbm_testing,Male,alzheimers_disease_state,13.0,7.4,21.1
4,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,bbbm_testing_and_treatment,Female,alzheimers_disease_state,29.0,22.7,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2395,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,baseline,Male,alzheimers_disease_state,591.4,497.0,733.1
2396,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,bbbm_testing,Female,alzheimers_disease_state,1598.2,1421.2,1823.5
2397,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,bbbm_testing,Male,alzheimers_disease_state,591.4,497.0,733.1
2398,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,bbbm_testing_and_treatment,Female,alzheimers_disease_state,1574.8,1408.6,1801.1


In [38]:
deaths_summary1 = ops.summarize_draws(deaths, 'mean').reset_index()
deaths_summary1

,age_group,artifact_path,entity,entity_type,event_year,measure,scenario,sex,sub_entity,value
0,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,baseline,Female,alzheimers_disease_state,29.0
1,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,baseline,Male,alzheimers_disease_state,13.0
2,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,bbbm_testing,Female,alzheimers_disease_state,29.0
3,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,bbbm_testing,Male,alzheimers_disease_state,13.0
4,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2040,deaths,bbbm_testing_and_treatment,Female,alzheimers_disease_state,29.0
...,...,...,...,...,...,...,...,...,...,...
2395,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,baseline,Male,alzheimers_disease_state,591.4
2396,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,bbbm_testing,Female,alzheimers_disease_state,1598.2
2397,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,bbbm_testing,Male,alzheimers_disease_state,591.4
2398,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2049,deaths,bbbm_testing_and_treatment,Female,alzheimers_disease_state,1574.8
